# Latent Consistency Model using Optimum-Intel OpenVINO
This notebook provides instructions how to run Latent Consistency Model (LCM) on Intel® Tiber™ Developer Cloud. It allows to setup standard Hugging Face diffusers pipeline and Optimum Intel pipeline optimized for Intel hardware including CPU and GPU. Running inference on CPU and GPU it is easy to compare performance and time required to generate an image for provided prompt. The notebook can be also used on other Intel hardware with minimal or no modifications.  

![](https://github.com/openvinotoolkit/openvino_notebooks/assets/10940214/1858dae4-72fd-401e-b055-66d503d82446)

Optimum Intel is an interface from Hugging Face between both diffusers and transformers libraries and various tools provided by Intel to accelerate pipelines on Intel hardware. It allows to perform quantization of the models hosted on Hugging Face.
In this notebook OpenVINO is used for AI-inference acceleration as a backend for Optimum Intel! 

For more details please refer to Optimum Intel repository
https://github.com/huggingface/optimum-intel

<img src="https://github.com/openvinotoolkit/openvino_notebooks/assets/105707993/a668529a-e1bd-46c6-9be4-1e6ca705c939"/>


LCMs are the next generation of generative models after Latent Diffusion Models (LDMs). They are proposed to overcome the slow iterative sampling process of Latent Diffusion Models (LDMs), enabling fast inference with minimal steps (from 2 to 4) on any pre-trained LDMs (e.g. Stable Diffusion). To read more about LCM please refer to https://latent-consistency-models.github.io/

#### Table of contents:
- [Prerequisites](#Prerequisites)
- [Full precision model on the CPU](#Using-full-precision-model-in-CPU-with-LatentConsistencyModelPipeline)
- [Running inference using Optimum Intel `OVLatentConsistencyModelPipeline`](#Running-inference-using-Optimum-Intel-OVLatentConsistencyModelPipeline)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/latent-consistency-models-image-generation/latent-consistency-models-optimum-demo.ipynb" />


### Prerequisites
[back to top ⬆️](#Table-of-contents:)

Install required packages

In [1]:
import sys
!{sys.executable} -m pip install -q "openvino>=2023.3.0"
!{sys.executable} -m pip install -q "onnx>=1.11.0"
!{sys.executable} -m pip install -q "optimum-intel[diffusers]@git+https://github.com/huggingface/optimum-intel.git" "accelerate" "ipywidgets" "torch>=2.1" "transformers>=4.33.0" --extra-index-url https://download.pytorch.org/whl/cpu
!{sys.executable} -m pip uninstall --yes "huggingface-hub"
!{sys.executable} -m pip install "huggingface-hub==0.24.2"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import sys
username = os.environ.get('USER')
user_bin_path = os.path.expanduser(f"/home/{username}/.local/bin")
sys.path.append(user_bin_path)
print(sys.path)

In [2]:
import warnings

warnings.filterwarnings("ignore")

### Showing Info Available Devices
[back to top ⬆️](#Table-of-contents:)

The `available_devices` property shows the available devices in your system. The "FULL_DEVICE_NAME" option to `ie.get_property()` shows the name of the device. 

Note: For more details about GPU with OpenVINO visit this [link](https://docs.openvino.ai/2024/get-started/configurations/configurations-intel-gpu.html). If you have been facing any issue in Ubuntu 20.04 or Windows 11 read this [blog](https://blog.openvino.ai/blog-posts/install-gpu-drivers-windows-ubuntu).

In [ ]:
import openvino as ov

core = ov.Core()
devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

### Using full precision model in CPU with `LatentConsistencyModelPipeline`
[back to top ⬆️](#Table-of-contents:)

Standard pipeline for the Latent Consistency Model(LCM) from Diffusers library is used here. For more information please refer to  https://huggingface.co/docs/diffusers/en/api/pipelines/latent_consistency_models


In [ ]:
from diffusers import LatentConsistencyModelPipeline
import gc

pipeline = LatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7")

In [ ]:
prompt = "A cute squirrel in the forest, portrait, 8k"

image = pipeline(prompt=prompt, num_inference_steps=4, guidance_scale=8.0, height=512, width=512).images[0]
image.save("image_standard_pipeline.png")
image

In [7]:
del pipeline
gc.collect();

345

### Select inference device for text-to-image generation

In [ ]:
import ipywidgets as widgets

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

device

### Running inference using Optimum Intel `OVLatentConsistencyModelPipeline`
[back to top ⬆️](#Table-of-contents:)

Accelerating inference of LCM using Intel Optimum with OpenVINO backend. For more information please refer to https://huggingface.co/docs/optimum/intel/inference#latent-consistency-models. 
The pretrained model in this notebook is available on Hugging Face in FP32 precision and in case if CPU is selected as a device, then inference runs with full precision. For GPU accelerated AI-inference is supported for FP16 data type and FP32 precision for GPU may produce high memory footprint and latency. Therefore, default precision for GPU in OpenVINO is FP16. OpenVINO GPU Plugin converts FP32 to FP16 on the fly and there is no need to do it manually

In [ ]:
from optimum.intel.openvino import OVLatentConsistencyModelPipeline
from pathlib import Path
ov_config = {"INFERENCE_PRECISION_HINT": "fp16"}

if not Path("./openvino_ir").exists():
    ov_pipeline = OVLatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7", height=512, width=512, export=True, compile=False, ov_config=ov_config)
    ov_pipeline.save_pretrained("./openvino_ir")
else:
    ov_pipeline = OVLatentConsistencyModelPipeline.from_pretrained("./openvino_ir", export=False, compile=False, ov_config=ov_config)

ov_pipeline.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)

In [ ]:
ov_pipeline.to(device.value)
ov_pipeline.compile()

In [ ]:
prompt = "A cute squirrel in the forest, portrait, 8k"

image_ov = ov_pipeline(prompt=prompt, num_inference_steps=4, guidance_scale=8.0, height=512, width=512).images[0]
image_ov.save("image_opt.png")
image_ov

In [ ]:
del ov_pipeline
gc.collect();